# Analysis

In der Phase des Prototyping haben wir vom Datenset erst die ersten 1'000 Zeilen verwendet. Nach den Erkenntnissen aus dem Prototyping laden wir nun die vollständigen Daten auf unseren Cluster.

Der verwendete Datensatz verfügt über zwei Files. Das grosse File steht als CSV-File zur Verfügung, das zweite (Carriers) wird als TXT-File hochgeladen.
Beim Prototyping hat sich gezeigt, dass die Datenmenge beim Speichern als Parquet signifikant reduziert hat. Somit können wir die Performance der Berechnungen verbessern, wenn wir unseren vollständigen Datensatz ebenfalls als Parquet speichern.

### Weshalb Parquet?

Unser Datensatz verfügt über ca. 12GB Daten. Da Parquet auf Geschwindigkeit und Komprimierung optimiert ist, eigent sich dieses Format für solch grosse Datenmengen. Auch verfügt unser Datensatz über 29 Spalten. Bei all unseren Fragestellungen benötigen wir jedoch nur immer einen Bruchteil der Spalten, weshalb das Auslesen der Daten (was immer Zeilenweise geschieht) schneller ist mit Parquet.
Auch haben wir viele identische "NA" Einträge, welche durch die Kompirimierung die Datenmenge signifikant reduziert.

## Bereitstellen der vollständigen Daten

Das Verzeichnis ist bereits aus dem Prototypin vorhanden. Wir werwenden  /airline-data.

In [10]:
!hdfs dfs -ls /airline-data

Found 2 items
drwxr-xr-x   - cluster supergroup          0 2022-05-12 12:35 /airline-data/airline.parquet
-rw-r--r--   2 cluster supergroup      43758 2022-05-12 12:29 /airline-data/carriers.txt


In [ ]:
# !hdfs dfs -put ~/BDLC-Project-G01/Dataset/airline.csv /airline-data
# !hdfs dfs -put ~/BDLC-Project-G01/Dataset/carriers.txt /airline-data

In [ ]:
!hdfs dfs -ls /airline-data

### 1. Daten als Parquet speichern

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('ConvertToParquet').getOrCreate()

In [ ]:
!hdfs dfs -ls /airline-data/

### a) Parquet File schreiben

In [ ]:
spark.read.csv('/airline-data/airline.csv', header=True).repartition(55).write.parquet('/airline-data/airline.parquet')

In [1]:
!hdfs dfs -ls /airline-data/

Found 2 items
drwxr-xr-x   - cluster supergroup          0 2022-05-12 12:35 /airline-data/airline.parquet
-rw-r--r--   2 cluster supergroup      43758 2022-05-12 12:29 /airline-data/carriers.txt


### b) Raw File löschen

In [ ]:
!hdfs dfs -rm /airline-data/airline.csv

In [ ]:
!hdfs dfs -ls /airline-data/

### c) Spark-Session stoppen

In [ ]:
spark.stop()

## Datensatz in Tabelle schreiben

In [ ]:
#!hdfs dfs -mkdir /user/spark/warehouse

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("SQL with SPARK") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-19 12:16:05,915 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


"spark"  ist nun eine existierende SparkSession

In dieser erstellen wir nun ein Dataframe, welches innerhalb dieses Spark-Session verfügbar ist:

In [11]:
df = spark.read.parquet("/airline-data/airline.parquet")

In [12]:
df.show(n=10)

2022-05-19 12:45:33,302 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+
|              120|     98|       4|   1214|      1210|       915|           115|            n

## Plausibilisierung des DataFrames

Vor der Bearbeitung der Fragestellungen muss der Datensatz zuerst noch plausibilisiert werden. Hierfür zählen wir die Datensätze:

In [16]:
df.createOrReplaceTempView("df")

In [17]:
anzahlZeilen = spark.sql("SELECT COUNT(FlightNum) FROM df")

In [18]:
anzahlZeilen.show()

+----------------+
|count(FlightNum)|
+----------------+
|       123534969|
+----------------+



Die Auswertung zeigt, dass im Datensatz 123.5 Mio Flugzeilen enthalten sind. Dies haben wir über den Datensatz lokal auf dem Laptop verifiziert (:-)= würde natürlich bei einem echten Big-Data-Problem nicht funktionieren.

# Fragestellungen

Die nachfolgenden Abfragen können somit nun in der SparkSession und auf dem DataFrame ausgewertet werden.

### Fluggesellschaften

#### Welche Fluggesellschaften haben prozentual (+/-) die grössten Abweichungen der geplanten Flugzeit zur effektiven Flugzeit (geplante Abflugzeit bis effektive Ankunftszeit)?

In [20]:
fluggesellschaften = spark.sql("SELECT UniqueCarrier, SUM(ArrDelay) FROM df GROUP BY UniqueCarrier")

In [21]:
fluggesellschaften.show()

+-------------+-------------+
|UniqueCarrier|sum(ArrDelay)|
+-------------+-------------+
|           UA| 1.10685605E8|
|           EA|    6279305.0|
|           PI|    9022518.0|
|           PS|     762188.0|
|           AA|  9.9540007E7|
|           NW|  5.5155194E7|
|           EV|  1.7671819E7|
|           B6|    7724044.0|
|           HP|  2.7049187E7|
|           TW|  2.5218176E7|
|           DL| 1.22853432E8|
|           OO|  1.7751788E7|
|           F9|    1906321.0|
|           YV|    9409631.0|
|           TZ|    1262537.0|
|           US|  8.8558778E7|
|           AQ|     175282.0|
|           MQ|  3.2432586E7|
|           OH|  1.0580107E7|
|           HA|    -211526.0|
+-------------+-------------+
only showing top 20 rows



#### Bei welchen Fluggesellschaften werden die meisten Flüge gestrichen?

#

### Flughäfen

#### Welche 10 Flughäfen haben die kleinste und grösste Abflugverspätung (ohne cancellation)?

#### Bei welchen Flughäfen werden die meisten Flüge gestrichen?

#

### Weitere Fragestellungen

#### An welchen Tagen gibt es am meisten Flugbewegungen?

#### Können Unregelmässigkeiten wie Naturkatastrophen oder Terroranschläge festgestellt werden? 